In [68]:
it = next(iter(range(3)))


0

In [84]:
it = iter(range(3))


In [85]:
next(it)

0

In [1]:
import numpy as np
import pandas as pd
import random
from collections import deque
import torch
import torch.nn as nn

In [ ]:
from neural import Net

class Agent:
    def __init__(self, state_size, is_eval=False, model_name=""):
        self.state_size = state_size  # normalized previous days
        self.action_size = 3  # hold, buy, sell
        self.memory = deque(maxlen=5000)
        self.model_name = model_name
        self.is_eval = is_eval

        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995

        self.use_cuda = torch.cuda.is_available()
        self.model = Net(self.state_size, self.action_size).float()
        
        if self.use_cuda:
            self.net = self.net.to(device='cuda')
        if is_eval:
            self.load(model_name)        
        
        self.model =  load if is_eval else 
        self.model_target = Net(self.state_size, self.action_size).float()
        
        self.model = load_model(model_name) if is_eval else self._model()
        self.model_target = load_model(model_name) if is_eval else self._model()
        self.update_target_from_model()
        self.loss = []




    def update_target_from_model(self):
        #Update the target model from the base model
        self.model_target.set_weights(self.model.get_weights())


    def act(self, state):
        if not self.is_eval and np.random.rand() <= self.epsilon:
            a1 = random.random()
            a2 = random.uniform(0, 1 - a1)
            a3 = 1 - a1 - a2
            return np.array([[a1, a2, a3]])

        options = self.model.predict(state)
        return softmax(options)

    def exp_replay(self, batch_size):

        minibatch = random.sample(self.memory, batch_size)

        states = np.array([tup[0][0] for tup in minibatch])
        actions = np.array([tup[1] for tup in minibatch])
        rewards = np.array([tup[2] for tup in minibatch])
        next_states = np.array([tup[3][0] for tup in minibatch])
        done = np.array([tup[4] for tup in minibatch])

        st_predict = self.model.predict(states)
        nst_predict = self.model.predict(next_states)
        nst_predict_target = self.model_target.predict(next_states)

        nst_predict_max_index = np.argmax(nst_predict, axis=1) # leanring agent의 Q값중 큰 action
        one_hot_max_index = tf.one_hot(nst_predict_max_index, self.action_size)

        target = rewards + self.gamma * np.amax(nst_predict_target*one_hot_max_index,axis=1) # 미래
        target[done] = rewards[done]

        target_f = st_predict
        target_f[range(batch_size), actions] = target

        # Q(s', a)
        #target = rewards + self.gamma * np.amax(self.model.predict(next_states), axis=1) #미래
        # end state target is reward itself (no lookahead)
        #target[done] = rewards[done]

        # Q(s, a)
        #target_f = self.model.predict(states) #현재로 예측하고? 사실상 array 만들어주는 역할, q(s,a)

        # make the agent to approximately map the current state to future discounted reward
        #target_f[range(batch_size), actions] = target #Q(s', a) 값을 업데이트, argmaxQ(s_t+1,a)

        hist = self.model.fit(states, target_f, epochs=1, verbose=0) #현재 스테이트 넣고 계산된 미래 Q값을 학습시키는 것
        #print(hist.history['loss'])
        #self.loss.append(hist.history['loss'][0]
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay


## nerual.py

In [ ]:
from torch import nn
import copy

class Net(nn.Module):

    def __init__(self, input_dim, output_dim):
        super().__init__()

        self.model = nn.Sequential(
            nn.Linear(input_dim,300)
            nn.ReLU(),
            nn.Linear(300,200)
            nn.ReLU(),
            nn.Linear(200,100)
            nn.ReLU(),
            nn.Linear(100, output_dim)
        )

        self.target = copy.deepcopy(self.online)

        # Q_target parameters are frozen.
        for p in self.target.parameters():
            p.requires_grad = False

    def forward(self, input, model):
        if model == 'online':
            return self.online(input)
        elif model == 'target':
            return self.target(input)        



## 통째로 바꿔보기 agent.py

In [ ]:
import torch
import random, numpy as np
from pathlib import Path

from neural import Net
from collections import deque
import torch.nn.functional as F

class Agent:
    def __init__(self, state_size, save_dir, checkpoint=None):
        self.state_dim = state_size
        self.action_dim = 3
        self.memory = deque(maxlen=10000)
        self.batch_size = 32

        self.exploration_rate = 1                  #epsilon
        self.exploration_rate_decay = 0.9995       #epsilon_dacay
        self.exploration_rate_min = 0.1            #epsilon_min
        self.gamma = 0.95

        self.curr_step = 0
        self.burnin = 100  # min. experiences before training
        self.learn_every = 3   # no. of experiences between updates to Q_online
        self.sync_every = 1000   # no. of experiences between Q_target & Q_online sync <???>

        self.save_every = 500   # no. of experiences between saving Agent
        self.save_dir = save_dir

        self.use_cuda = torch.cuda.is_available()

        # Mario's DNN to predict the most optimal action - we implement this in the Learn section
        self.net = Net(self.state_dim, self.action_dim).float()
        if self.use_cuda: #쿠다 사용여부
            self.net = self.net.to(device='cuda')
        if checkpoint:
            self.load(checkpoint)

        self.optimizer = torch.optim.Adam(self.net.parameters(), lr=0.00025)
        self.loss_fn = torch.nn.MSELoss() #MSE로 로스 펑션 사용, RMSE도 고려해볼만하다.


    def act(self, state):

        # EXPLORE
        #if np.random.rand() < self.exploration_rate:
        #    action_idx = np.random.randint(self.action_dim)
            
        if np.random.rand() < self.exploration_rate:
            a1 = random.random()
            a2 = random.uniform(0, 1 - a1)
            a3 = 1 - a1 - a2
            action_values = torch.from_numpy(np.array([[a1, a2, a3]]))    

        # EXPLOIT
        else:
            state = torch.FloatTensor(state).cuda() if self.use_cuda else torch.FloatTensor(state)
            #state = state.unsqueeze(0)
            action_values = F.softmax(self.net(state, model='online'),dim=0) #모델을 거쳤을 때 각 행동의 기대값 ex) 3가지 액션일때 [[0.1, 0.3, 0.6]]
           
        action_idx = torch.argmax(action_values, axis=1).item()
        
        # decrease exploration_rate
        self.exploration_rate *= self.exploration_rate_decay
        self.exploration_rate = max(self.exploration_rate_min, self.exploration_rate)

        # increment step
        self.curr_step += 1
        return torch.max(action_values), action_idx
        
    
    def cache(self, state, next_state, action, reward, done):
        """
        Store the experience to self.memory (replay buffer)
        Inputs:
        state (LazyFrame),
        next_state (LazyFrame),
        action (int),
        reward (float),
        done(bool))
        """
        state = torch.FloatTensor(state).cuda() if self.use_cuda else torch.FloatTensor(state)
        next_state = torch.FloatTensor(next_state).cuda() if self.use_cuda else torch.FloatTensor(next_state)
        action = torch.LongTensor([action]).cuda() if self.use_cuda else torch.LongTensor([action])
        reward = torch.DoubleTensor([reward]).cuda() if self.use_cuda else torch.DoubleTensor([reward])
        done = torch.BoolTensor([done]).cuda() if self.use_cuda else torch.BoolTensor([done])

        self.memory.append( (state, next_state, action, reward, done,) )


    def recall(self):
        """
        Retrieve a batch of experiences from memory
        """
        batch = random.sample(self.memory, self.batch_size)
        state, next_state, action, reward, done = map(torch.stack, zip(*batch))
        return state, next_state, action.squeeze(), reward.squeeze(), done.squeeze()


    def td_estimate(self, state, action):
        current_Q = self.net(state, model='online')[np.arange(0, self.batch_size), action] # Q_online(s,a)
        return current_Q


    @torch.no_grad()
    def td_target(self, reward, next_state, done):
        next_state_Q = self.net(next_state)
        best_action = torch.argmax(next_state_Q, axis=1)
        next_Q = self.net(next_state, model='target')[np.arange(0, self.batch_size), best_action]
        return (reward + (1 - done.float()) * self.gamma * next_Q).float()


    def update_Q_online(self, td_estimate, td_target) :
        loss = self.loss_fn(td_estimate, td_target)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        return loss.item()


    def sync_Q_target(self):
        self.net.target.load_state_dict(self.net.online.state_dict())


    def learn(self):
        if self.curr_step % self.sync_every == 0:
            self.sync_Q_target()

        if self.curr_step % self.save_every == 0:
            self.save()

        if self.curr_step < self.burnin:
            return None, None

        if self.curr_step % self.learn_every != 0:
            return None, None

        # Sample from memory
        state, next_state, action, reward, done = self.recall()

        # Get TD Estimate
        td_est = self.td_estimate(state, action)

        # Get TD Target
        td_tgt = self.td_target(reward, next_state, done)

        # Backpropagate loss through Q_online
        loss = self.update_Q_online(td_est, td_tgt)

        return (td_est.mean().item(), loss)


    def save(self):
        save_path = self.save_dir / f"mario_net_{int(self.curr_step // self.save_every)}.chkpt"
        torch.save(
            dict(
                model=self.net.state_dict(),
                exploration_rate=self.exploration_rate
            ),
            save_path
        )
        print(f"MarioNet saved to {save_path} at step {self.curr_step}")


    def load(self, load_path):
        if not load_path.exists():
            raise ValueError(f"{load_path} does not exist")

        ckp = torch.load(load_path, map_location=('cuda' if self.use_cuda else 'cpu'))
        exploration_rate = ckp.get('exploration_rate')
        state_dict = ckp.get('model')

        print(f"Loading model at {load_path} with exploration rate {exploration_rate}")
        self.net.load_state_dict(state_dict)
        self.exploration_rate = exploration_rate

## metrics.py

In [ ]:
import numpy as np
import time, datetime
import matplotlib.pyplot as plt

class MetricLogger():
    def __init__(self, save_dir):
        self.save_log = save_dir / "log"
        with open(self.save_log, "w") as f:
            f.write(
                f"{'Episode':>8}{'Step':>8}{'Epsilon':>10}{'MeanReward':>15}"
                f"{'MeanLength':>15}{'MeanLoss':>15}{'MeanQValue':>15}"
                f"{'TimeDelta':>15}{'Time':>20}\n"
            )
        self.ep_rewards_plot = save_dir / "reward_plot.jpg"
        self.ep_lengths_plot = save_dir / "length_plot.jpg"
        self.ep_avg_losses_plot = save_dir / "loss_plot.jpg"
        self.ep_avg_qs_plot = save_dir / "q_plot.jpg"

        # History metrics
        self.ep_rewards = []
        self.ep_lengths = []
        self.ep_avg_losses = []
        self.ep_avg_qs = []

        # Moving averages, added for every call to record()
        self.moving_avg_ep_rewards = []
        self.moving_avg_ep_lengths = []
        self.moving_avg_ep_avg_losses = []
        self.moving_avg_ep_avg_qs = []

        # Current episode metric
        self.init_episode()

        # Timing
        self.record_time = time.time()


    def log_step(self, reward, loss, q):
        self.curr_ep_reward += reward
        self.curr_ep_length += 1
        if loss:
            self.curr_ep_loss += loss
            self.curr_ep_q += q
            self.curr_ep_loss_length += 1

    def log_episode(self):
        "Mark end of episode"
        self.ep_rewards.append(self.curr_ep_reward)
        self.ep_lengths.append(self.curr_ep_length)
        if self.curr_ep_loss_length == 0:
            ep_avg_loss = 0
            ep_avg_q = 0
        else:
            ep_avg_loss = np.round(self.curr_ep_loss / self.curr_ep_loss_length, 5)
            ep_avg_q = np.round(self.curr_ep_q / self.curr_ep_loss_length, 5)
        self.ep_avg_losses.append(ep_avg_loss)
        self.ep_avg_qs.append(ep_avg_q)

        self.init_episode()

    def init_episode(self):
        self.curr_ep_reward = 0.0
        self.curr_ep_length = 0
        self.curr_ep_loss = 0.0
        self.curr_ep_q = 0.0
        self.curr_ep_loss_length = 0

    def record(self, episode, epsilon, step):
        mean_ep_reward = np.round(np.mean(self.ep_rewards[-100:]), 3)
        mean_ep_length = np.round(np.mean(self.ep_lengths[-100:]), 3)
        mean_ep_loss = np.round(np.mean(self.ep_avg_losses[-100:]), 3)
        mean_ep_q = np.round(np.mean(self.ep_avg_qs[-100:]), 3)
        self.moving_avg_ep_rewards.append(mean_ep_reward)
        self.moving_avg_ep_lengths.append(mean_ep_length)
        self.moving_avg_ep_avg_losses.append(mean_ep_loss)
        self.moving_avg_ep_avg_qs.append(mean_ep_q)


        last_record_time = self.record_time
        self.record_time = time.time()
        time_since_last_record = np.round(self.record_time - last_record_time, 3)

        print(
            f"Episode {episode} - "
            f"Step {step} - "
            f"Epsilon {epsilon} - "
            f"Mean Reward {mean_ep_reward} - "
            f"Mean Length {mean_ep_length} - "
            f"Mean Loss {mean_ep_loss} - "
            f"Mean Q Value {mean_ep_q} - "
            f"Time Delta {time_since_last_record} - "
            f"Time {datetime.datetime.now().strftime('%Y-%m-%dT%H:%M:%S')}"
        )

        with open(self.save_log, "a") as f:
            f.write(
                f"{episode:8d}{step:8d}{epsilon:10.3f}"
                f"{mean_ep_reward:15.3f}{mean_ep_length:15.3f}{mean_ep_loss:15.3f}{mean_ep_q:15.3f}"
                f"{time_since_last_record:15.3f}"
                f"{datetime.datetime.now().strftime('%Y-%m-%dT%H:%M:%S'):>20}\n"
            )

        for metric in ["ep_rewards", "ep_lengths", "ep_avg_losses", "ep_avg_qs"]:
            plt.plot(getattr(self, f"moving_avg_{metric}"))
            plt.savefig(getattr(self, f"{metric}_plot"))
            plt.clf()

## Main.py

In [ ]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

import random, datetime
from pathlib import Path

from metrics import MetricLogger
from agent import Agent
from Dataset import CoinDataset
from env import env

# Initialize Super Mario environment
trade_env =  env(learning_length=2400,"out_data.csv")#데이터 불러오기


save_dir = Path('checkpoints') / datetime.datetime.now().strftime('%Y-%m-%dT%H-%M-%S')
save_dir.mkdir(parents=True)

checkpoint = None # Path('checkpoints/2020-10-21T18-25-27/mario.chkpt')

state_size = trade_env.state_data.shape[1]
    
trade_agent = Agent(state_dim=state_size, action_dim=3, save_dir=save_dir, checkpoint=checkpoint)

logger = MetricLogger(save_dir)

episodes = 40000

### for Loop that train the model num_episodes times by playing the game
for e in range(episodes):
    
    #state = env.reset()
    state = trade_env.state_data[0]
    L_num = 100
    prev_num = 0
    now_length = 0
    

    # Play the game!
    while True:

        # 3. Show environment (the visual) [WIP]
        # env.render()

        # 4. Run agent on the state
        q_value, action = trade_agent.act(state)
        
        
        # 5. Agent performs action
        next_state, reward, done, now_length = trade_env.step(action, q_value, L_num, prev_num, now_length, done)

        # 6. Remember
        trade_agent.cache(state, next_state, action, reward, done)

        # 7. Learn
        q, loss = trade_agent.learn()

        # 8. Logging
        logger.log_step(reward, loss, q)

        # 9. Update state
        state = next_state

        # 10. Check if end of game
        if done :
            break

    logger.log_episode()

    if e % 20 == 0:
        logger.record(
            episode=e,
            epsilon=mario.exploration_rate,
            step=mario.curr_step
        )

## Dataset.py

In [47]:
import os
import pandas as pd
import torch
import numpy
from torch.utils.data import Dataset, DataLoader

class CoinDataset(Dataset):
    def __init__(self, file_name):
        self.raw_data = pd.read_csv(file_name)
        self.state_data = torch.from_numpy(raw_data.iloc[:,1:].values)
        self.price_data = torch.from_numpy(raw_data["Price"])

    def __len__(self):
        return len(self.state_data)

    def __getitem__(self, idx):
        return self.state_data[idx].unsqueeze(0), self.price_data[idx]



## env.py

In [ ]:
import torch
import numpy as np
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from Dataset import CoinDataset

class env():
    def __init__(self, learning_length, file_name):
        self.dataset = torch.utils.data.DataLoader(CoinDataset(file_name), batch_size=learning_length)
        self.state_data = self.dataset.state_data
        self.price_data = self.dataset.price_data
        
    def step(self, action,q_value, L_num, prev_num, now_length, done):
        reward = 0.0
        price_t0 = self.price_data[now_length]
        now_length += 1
        price_t1 = self.price_data[now_length]
        price_diff = price_t1-price_t0
        
        num=q_value*L_num
        num_diff = num - prev_num
        reward = num_diff * price_diff
        
        next_state = self.state_data[now_length]
        
        if now_length == learning_length :
            done = True
            
        return next_state, reward, done, now_length

## replay.py

In [ ]:
import random, datetime
from pathlib import Path

import gym
import gym_super_mario_bros
from gym.wrappers import FrameStack, GrayScaleObservation, TransformObservation
from nes_py.wrappers import JoypadSpace

from metrics import MetricLogger
from agent import Mario
from wrappers import ResizeObservation, SkipFrame

# Initialize Super Mario environment
trade_env = env(learning_length=2502, file_name="out_data.csv")  # 데이터 불러오기

save_dir = Path('checkpoints') / datetime.datetime.now().strftime('%Y-%m-%dT%H-%M-%S')
save_dir.mkdir(parents=True)

checkpoint = None  # Path('checkpoints/2020-10-21T18-25-27/mario.chkpt')

state_data, price_data = next(iter(trade_env.data))

trade_agent = Agent(state_size=3, save_dir=save_dir, checkpoint=checkpoint)

logger = MetricLogger(save_dir)

episodes = 2000
trade_agnet.exploration_rate = trade_agnet.exploration_rate_min

for e in range(episodes):

    state = env.reset()

    while True:


        action = mario.act(state)

        next_state, reward, done, info = env.step(action)

        mario.cache(state, next_state, action, reward, done)

        logger.log_step(reward, None, None)

        state = next_state

        if done or info['flag_get']:
            break

    logger.log_episode()

    if e % 20 == 0:
        logger.record(
            episode=e,
            epsilon=mario.exploration_rate,
            step=mario.curr_step
        )
        
for e in range(episodes):

    state = trade_env.reset()
    L_num = 100
    prev_num = 0
    trade_agent.curr_step = 0

    # Play the game!
    while True:


        # 4. Run agent on the state
        q_value, action = trade_agent.act(state) #여기서 curr_step이 1올라감

        # 5. Agent performs action
        next_state, reward, total_profit, done = trade_env.step(action, q_value, L_num, prev_num, trade_agent.curr_step)

        # 6. Remember
        trade_agent.cache(state, next_state, action, reward, done)

        # 8. Logging
        logger.log_step(reward.detach().cpu().numpy(), None, None)

        # 9. Update state
        state = next_state

        # 10. Check if end of game
        if done:
            break

    logger.log_episode()

    if e % 20 == 0:
        logger.record(
            episode=e,
            epsilon=trade_agent.exploration_rate,
            step=trade_agent.curr_step
        )

In [ ]:
import os

os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

import random, datetime
from pathlib import Path

from metrics import MetricLogger
from agent import Agent
from env import env

# Initialize Super Mario environment
trade_env = env(learning_length=2502, file_name="out_data.csv")  # 데이터 불러오기

save_dir = Path('checkpoints') / datetime.datetime.now().strftime('%Y-%m-%dT%H-%M-%S')
save_dir.mkdir(parents=True)

checkpoint = None  # Path('checkpoints/2020-10-21T18-25-27/mario.chkpt')

state_size, _ = trade_env.coindata[1] #하나 부를땐 이거
state_data, price_data = next(iter(trade_env.data)) #데이터 로더에 넣어서 부를땐??

trade_agent = Agent(state_size=3, save_dir=save_dir, checkpoint=checkpoint)

logger = MetricLogger(save_dir)

episodes = 2000

### for Loop that train the model num_episodes times by playing the game
for e in range(episodes):

    state = trade_env.reset()
    L_num = 100
    prev_num = 0
    trade_agent.curr_step = 0

    # Play the game!
    while True:

        # 3. Show environment (the visual) [WIP]
        # env.render()

        # 4. Run agent on the state
        q_value, action = trade_agent.act(state) #여기서 curr_step이 1올라감

        # 5. Agent performs action
        next_state, reward, total_profit, done = trade_env.step(action, q_value, L_num, prev_num, trade_agent.curr_step)

        # 6. Remember
        trade_agent.cache(state, next_state, action, reward, done)

        # 7. Learn
        q, loss = trade_agent.learn()

        # 8. Logging
        logger.log_step(reward.detach().cpu().numpy(), loss, q)

        # 9. Update state
        state = next_state

        # 10. Check if end of game
        if done:
            break

    logger.log_episode()

    if e % 10 == 0:
        logger.record(
            episode=e,
            epsilon=trade_agent.exploration_rate,
            step=trade_agent.curr_step
        )

In [87]:
import numpy as np
import torch
import pandas as pd
a = np.random.rand(10,4)

In [88]:
a=pd.DataFrame(a)
k=a.iloc[1:,1:].values

In [89]:
a.shape

(10, 4)

In [90]:
a

,0,1,2,3
0,0.168702,0.444208,0.340190,0.408997
1,0.199491,0.547270,0.777295,0.443051
2,0.944127,0.615175,0.782584,0.703338
3,0.453528,0.540171,0.556252,0.541182
4,0.843560,0.007810,0.888476,0.183707
5,0.361536,0.967983,0.557054,0.390935
6,0.713508,0.142903,0.649832,0.250312
7,0.616661,0.138813,0.634591,0.042524
8,0.193803,0.233025,0.184401,0.003505
9,0.959506,0.508941,0.933035,0.354339


In [98]:
k

array([[0.54727008, 0.77729497, 0.44305057],
       [0.61517515, 0.78258445, 0.70333836],
       [0.54017053, 0.55625197, 0.54118164],
       [0.00780967, 0.88847635, 0.18370694],
       [0.96798262, 0.55705403, 0.39093511],
       [0.14290295, 0.64983176, 0.25031229],
       [0.13881277, 0.6345908 , 0.04252441],
       [0.23302539, 0.18440119, 0.00350493],
       [0.50894118, 0.93303489, 0.3543395 ]])

In [109]:
torch.from_numpy(k).shape


torch.Size([9, 3])

In [21]:
b=torch.from_numpy(a)

In [24]:
b.reshape(1,-1)

tensor([[0.0595, 0.9188, 0.1970, 0.1810, 0.7091, 0.7207, 0.1504, 0.2708, 0.1461,
         0.0186, 0.1998, 0.9948, 0.4809, 0.7254, 0.5702, 0.8193, 0.1620, 0.5140,
         0.0561, 0.7193, 0.5418, 0.6215, 0.0038, 0.8768, 0.1731, 0.6158, 0.8829,
         0.7870, 0.6645, 0.0460]], dtype=torch.float64)

In [50]:
b.shape[0]

10

In [15]:
a[0].reshape(1,-1)

array([[0.297115  , 0.50277847, 0.515187  ]])

In [16]:
a[0]

array([0.297115  , 0.50277847, 0.515187  ])

In [12]:
a.reshape(1,-1)

array([[0.297115  , 0.50277847, 0.515187  , 0.20275523, 0.5190563 ,
        0.39580412, 0.01170525, 0.33156668, 0.38176398, 0.08626995,
        0.97852628, 0.94416906, 0.31826684, 0.06503326, 0.39888082,
        0.3815041 , 0.58378501, 0.72640227, 0.59115665, 0.87793077,
        0.12857936, 0.06005596, 0.12663981, 0.16042522, 0.65099473,
        0.01920423, 0.19659542, 0.70084941, 0.67971793, 0.63033119]])

In [ ]:



        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        
        self.total_profit = torch.FloatTensor(0).to(self.device)
        self.reward = torch.FloatTensor(0).to(self.device)
        self.price_diff = torch.FloatTensor(0).to(self.device)
        self.num_diff = torch.FloatTensor(0).to(self.device)
        self.num = torch.FloatTensor(0).to(self.device)
        self.action = torch.FloatTensor(0).to(self.device)


    def reset(self):
        self.total_profit = 0

        return self.state_data[0]

    #@torch.no_grad()
    def step(self, action, q_value, L_num, prev_num, curr_step):
        
        self.action = torch.FloatTensor(action).to(self.device)
        done = torch.bool(False).to(self.device)
        #price_t0 = self.price_data[curr_step-1]
        #price_t1 = self.price_data[curr_step]

        self.price_diff = self.price_data[curr_step] - self.price_data[curr_step-1]

        #action 에따라 -,+ 바꿔줘야함 action[0,1,2]
        if self.action ==  torch.FloatTensor(1).to(self.device):
            self.num =  torch.FloatTensor(0).to(self.device)

        else:
            if self.action ==  torch.FloatTensor(0).to(self.device): #매도
                self.action =  torch.FloatTensor(-1).to(self.device)

            if self.action ==  torch.FloatTensor(2).to(self.device): #매수
                self.action =  torch.FloatTensor(1).to(self.device)

            self.num = action * q_value * L_num

        self.num_diff = prev_num + self.num
        self.reward = self.num_diff * self.price_diff
        self.total_profit += self.reward
        next_state = self.state_data[curr_step+1]


        if curr_step == self.learning_length-2:
            done = True

        return next_state, self.reward, self.total_profit, self.num, done


In [ ]:
self.device = "cuda" if torch.cuda.is_available() else "cpu"

self.total_profit = torch.FloatTensor(0).to(self.device)
self.reward = torch.FloatTensor(0).to(self.device)
self.price_diff = torch.FloatTensor(0).to(self.device)
self.num_diff = torch.FloatTensor(0).to(self.device)
self.num = torch.FloatTensor(0).to(self.device)
self.action = torch.FloatTensor(0).to(self.device)

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
device

'cpu'

In [16]:
reward = torch.FloatTensor([1]).to(device)

In [17]:
reward

tensor([1.])

In [18]:
if reward == 1:
    print("true")

true


In [19]:
reward

tensor([1.])

In [48]:
it = iter(range(3))


In [49]:
for i in range(2):
    k=next(it)

In [52]:
from torch.utils.data import TensorDataset # 텐서데이터셋
from torch.utils.data import DataLoader # 데이터로더
x_train  =  torch.FloatTensor([[73,  80,  75], 
                               [93,  88,  93], 
                               [89,  91,  90], 
                               [96,  98,  100],   
                               [73,  66,  70]])  
y_train  =  torch.FloatTensor([[152],  [185],  [180],  [196],  [142]])

dataset = TensorDataset(x_train, y_train)


In [53]:
dataloader = DataLoader(dataset, batch_size=2)


In [60]:
next(iter(dataloader))

[tensor([[73., 80., 75.],
         [93., 88., 93.]]),
 tensor([[152.],
         [185.]])]

In [62]:
for i in range(2):
    for a,b in dataloader:
        print(a,b)

tensor([[73., 80., 75.],
        [93., 88., 93.]]) tensor([[152.],
        [185.]])
tensor([[ 89.,  91.,  90.],
        [ 96.,  98., 100.]]) tensor([[180.],
        [196.]])
tensor([[73., 66., 70.]]) tensor([[142.]])
tensor([[73., 80., 75.],
        [93., 88., 93.]]) tensor([[152.],
        [185.]])
tensor([[ 89.,  91.,  90.],
        [ 96.,  98., 100.]]) tensor([[180.],
        [196.]])
tensor([[73., 66., 70.]]) tensor([[142.]])


In [59]:
a,b

(tensor([[73., 80., 75.],
         [93., 88., 93.]]),
 tensor([[152.],
         [185.]]))

In [81]:
for batch_idx, samples in enumerate(dataloader):
    if batch_idx == 0:
        break

In [82]:
batch_idx

0

In [88]:
samples

[tensor([[73., 80., 75.],
         [93., 88., 93.]]),
 tensor([[152.],
         [185.]])]